In [2]:
from tensorflow import keras
import xgboost as xgb, pandas as pd, spacy, numpy as np

2024-11-17 23:23:06.907797: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 23:23:06.908330: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 23:23:06.910436: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 23:23:06.917009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731896586.927648  601794 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731896586.93

In [3]:
nn_class_model = keras.models.load_model('models/best_classification_model.keras')
nn_reg_model = keras.models.load_model('models/best_regression_model.keras')

xg_model = xgb.XGBRegressor()
xg_model.load_model("models/final_xgboost_model.bin")

nlp = spacy.load('en_core_web_md')

2024-11-17 23:23:08.732702: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
headlines = pd.read_csv('data/new_headlines.csv')

headlines.head()

,fortnight,title,diff
0,4,FTSE falters ahead of Bernanke testimony HSBC ...,35.21
1,5,Northern Rock: more than 4% of mortgage custom...,28.56
2,6,The man who blew the whistle on Bernard Madoff...,21.72
3,7,Hedge fund group Man beats falling FTSE after ...,24.72
4,8,FTSE 100 slips but Arm and Autonomy move highe...,-32.34


In [5]:
vectors = headlines['title'].apply(lambda x: nlp(x).vector)

vectors_df = pd.DataFrame(vectors.tolist())

In [6]:
headlines = pd.concat([headlines.drop(columns=['title']), vectors_df], axis=1)

headlines = headlines[[column for column in headlines.columns if column != 'diff'] + ['diff']]

headlines.head()

,fortnight,0,1,2,3,4,5,6,7,8,...,291,292,293,294,295,296,297,298,299,diff
0,4,-1.449997,-0.077364,-2.283370,0.963611,3.524460,-0.222675,0.966745,3.083658,0.012949,...,-0.485189,-0.543872,0.530988,-1.232898,1.518342,0.949847,-1.350959,-1.443373,1.475914,35.21
1,5,-1.269134,-0.116923,-1.888452,0.715081,3.177433,-0.165495,1.057632,2.925950,-0.234037,...,-0.406570,-0.435821,0.315584,-1.270179,1.466113,0.994531,-1.320085,-1.598253,1.719888,28.56
2,6,-1.139919,-0.082824,-1.658028,0.859134,3.420564,-0.128764,1.016396,2.776837,-0.179581,...,-0.497938,-0.506435,0.563727,-1.271737,1.437843,0.852081,-1.241506,-1.565912,1.699312,21.72
3,7,-1.413767,-0.225218,-1.902161,1.095649,3.375605,-0.269922,0.886650,3.180560,-0.072454,...,-0.337137,-0.425653,0.401852,-1.357864,1.473602,1.009083,-1.310209,-1.639125,1.529785,24.72
4,8,-1.257028,-0.263146,-1.909343,0.864015,3.141049,-0.143166,1.186161,2.904674,-0.034254,...,-0.488059,-0.430336,0.532479,-1.078307,1.500332,0.943603,-1.325442,-1.362844,1.818235,-32.34


In [7]:
X, y_class, y_reg = headlines.drop(columns=['diff']), headlines['diff'].apply(lambda diff: 1 if diff > 0 else 0), headlines['diff']

In [8]:
nn = nn_reg_model.predict(X)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [9]:
stocks = pd.read_csv('data/fortnight_time_series.csv')

stocks.head()

,fortnight,S&P500_mean,Interest Rate_mean,GDP_mean,Unemployment Rate_mean,CPI_mean,Inflation Expectations_mean,Yield Spread_mean,VIX_mean,Industrial Production_mean,...,S&P500_mean_lag3,Interest Rate_mean_lag2,Interest Rate_mean_lag4,VIX_mean_lag1,VIX_mean_lag2,VIX_mean_lag3,GDP_mean_lag2,GDP_mean_lag4,Unemployment Rate_mean_lag2,Unemployment Rate_mean_lag4
0,4.0,1120.610000,0.141000,14764.61,9.87,217.329000,1.854000,2.806,18.887000,89.948000,...,1123.958889,0.130000,0.105556,22.002222,24.186000,21.021111,14764.61,14739.418889,9.80,9.822222
1,5.0,1158.773000,0.175000,14764.61,9.90,217.350000,1.894000,2.720,17.362000,90.140000,...,1080.225000,0.122222,0.118889,18.887000,22.002222,24.186000,14764.61,14764.610000,9.80,9.800000
2,6.0,1174.548889,0.165556,14836.47,9.90,217.366667,1.881111,2.820,17.416667,90.246667,...,1092.780000,0.141000,0.130000,17.362000,18.887000,22.002222,14764.61,14764.610000,9.87,9.800000
3,7.0,1197.618000,0.199000,14980.19,9.90,217.400000,1.952000,2.807,16.393000,90.460000,...,1120.610000,0.175000,0.122222,17.416667,17.362000,18.887000,14764.61,14764.610000,9.90,9.800000
4,8.0,1198.834000,0.200000,14980.19,9.84,217.378000,1.990000,2.726,19.529000,90.708000,...,1158.773000,0.165556,0.141000,16.393000,17.416667,17.362000,14836.47,14764.610000,9.90,9.870000


In [10]:
xg_model.get_booster().feature_names

['S&P500_mean',
 'Interest Rate_mean',
 'GDP_mean',
 'Unemployment Rate_mean',
 'CPI_mean',
 'Inflation Expectations_mean',
 'Yield Spread_mean',
 'VIX_mean',
 'Industrial Production_mean',
 'Consumer Confidence_mean',
 'Hourly Earnings_mean',
 'NFCI_mean',
 'Savings Rate_mean',
 'PCE_mean',
 'Money Supply_mean',
 'PPI_mean',
 'S&P500_std',
 'Interest Rate_std',
 'GDP_std',
 'Unemployment Rate_std',
 'CPI_std',
 'Inflation Expectations_std',
 'Yield Spread_std',
 'VIX_std',
 'Industrial Production_std',
 'Consumer Confidence_std',
 'Hourly Earnings_std',
 'NFCI_std',
 'Savings Rate_std',
 'PCE_std',
 'Money Supply_std',
 'PPI_std',
 'S&P500_mean_lag1',
 'S&P500_mean_lag2',
 'S&P500_mean_lag3',
 'Interest Rate_mean_lag2',
 'Interest Rate_mean_lag4',
 'VIX_mean_lag1',
 'VIX_mean_lag2',
 'VIX_mean_lag3',
 'GDP_mean_lag2',
 'GDP_mean_lag4',
 'Unemployment Rate_mean_lag2',
 'Unemployment Rate_mean_lag4']

In [11]:
X = stocks.drop(columns=['diff_Target', 'S&P500_Open_Price', 'S&P500_Close_Price', 'fortnight'])

xg = xg_model.predict(X)

In [29]:
predict = list(zip(xg, nn.flatten()))

final = [0.6 * xg + 0.4 * nn for xg, nn in predict]

final = list(zip(range(4, len(final)), final))

In [41]:
a = list(zip(final, headlines['diff']))

a = [(i[0], i[1], j) for i, j in a]

df_a = pd.DataFrame(a, columns=['fortnight', 'predict_diff', 'real_diff'])

df_a.head()

,fortnight,predict_diff,real_diff
0,4,2.391614,35.21
1,5,-0.546072,28.56
2,6,2.748283,21.72
3,7,-23.012702,24.72
4,8,-33.240987,-32.34


In [42]:
aa= stocks[['fortnight', 'S&P500_Open_Price', 'S&P500_Close_Price']]

new = df_a.merge(aa, how='left', on='fortnight')
new

,fortnight,predict_diff,real_diff,S&P500_Open_Price,S&P500_Close_Price
0,4,2.391614,35.21,1105.24,1140.45
1,5,-0.546072,28.56,1145.61,1174.17
2,6,2.748283,21.72,1167.72,1189.44
3,7,-23.012702,24.72,1182.45,1207.17
4,8,-33.240987,-32.34,1205.94,1173.60
...,...,...,...,...,...
373,377,32.929544,48.27,5421.03,5469.30
374,378,41.001853,99.08,5477.90,5576.98
375,379,50.549864,-78.17,5633.91,5555.74
376,380,63.293584,-187.10,5427.13,5240.03


In [44]:
new['predict_diff'] = new['predict_diff'].round(2)

In [46]:
new.to_csv('backtest.csv', index=False)